### Steps and tips
* Take sentences, apply one hot representation, pass through embedding layer, pass thorugh LSTM and then train the model to predict test data
* pad_sequences - input to LSTM / Embedding layer we have to make sure to keep our size input length fixed so we have to add pad either in the start or end
* one_hot - Convert sentences to one hot representation given a vocabulary size
* vocabulary size = 5000
* Onehot representation
* reset_index - since we dropped NaN values
* Data preprocessing - nltk, re, nltk.corpus - stopwords
* Data preprocessing - nltk.stem.porter - PorterStemmer (stemming is done using these libraries)
* re.sub('[^a-zA-Z]', ' ', messages['title'][i]) - we are substituting all the characters excep a-z and A-Z with a blank space
* then lowering of the whole review and then split to remove the stopwords as they are not important
* then all the text data is converted into one hot representation
* Embedding representation
* Before giving to embedding layer we will have to make the size of all inputs same, hence padding (pad_sequence)
* embedding_vector_features = 40
* Sequential - Embedding - LSTM - Dense (AF = sigmoid) - compile (binary_crossentropy, adam, accuracy)
* embedded docs are converted into an array
* train test split on X_final and y_final
* Model training (sign of overfitting since val_accuracy was less than accuracy of training data), adding a dropout
* performance metrics and accuracy
* adding the dropout layer is just a kind of hyperparameter to increase accuracy

In [1]:
! pip install kaggle

In [10]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [11]:
! cp kaggle.json ~/.kaggle/

In [12]:
! chmod 600 ~/.kaggle/kaggle.json

In [13]:
! kaggle competitions download fake-news

 88% 41.0M/46.5M [00:00<00:00, 68.2MB/s]
100% 46.5M/46.5M [00:00<00:00, 65.7MB/s]


In [14]:
! unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [15]:
! ls

fake-news.zip  kaggle.json  sample_data  submit.csv  test.csv  train.csv


Main code starts below -

In [16]:
import pandas as pd

In [50]:
df = pd.read_csv('/content/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [51]:
# Droping null / NaN values if any
df = df.dropna()

In [52]:
# Extracting independent features
X = df.drop('label', axis = 1)

In [53]:
# Dependent features
y = df['label']

In [54]:
X.shape, y.shape

((18285, 4), (18285,))

In [22]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [23]:
#importing all important libraries
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [24]:
voc_size  = 5000

Onehot Representation

In [25]:
messages = X.copy()

In [26]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [27]:
messages.reset_index(inplace = True)

In [28]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Data preprocessing

In [55]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [56]:
corpus[0:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [57]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr[0:5]

[[2785, 2100, 835, 1124, 152, 38, 4189, 1611, 678, 3443],
 [2131, 3890, 1665, 2830, 4034, 4871, 4788],
 [1031, 4896, 645, 3026],
 [4981, 1306, 282, 1416, 4663, 1847],
 [655, 4034, 242, 773, 585, 1231, 4034, 1117, 2785, 1869]]

Embedding representation

In [58]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)
embedded_docs[0:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2785,
        2100,  835, 1124,  152,   38, 4189, 1611,  678, 3443],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 2131, 3890, 1665, 2830, 4034, 4871, 4788],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 1031, 4896,  645, 3026],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 4981, 1306,  282, 1416, 4663, 1847],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  655,
        4034,  242,  773,  585, 1231, 4034, 1117, 2785, 1869]],
      dtype=int32)

In [59]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2785,
       2100,  835, 1124,  152,   38, 4189, 1611,  678, 3443], dtype=int32)

Model creation

In [60]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [61]:
len(embedded_docs)

18285

In [62]:
y.shape

(18285,)

In [63]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [64]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model training

In [66]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 15s 63ms/step - loss: 0.3210 - accuracy: 0.8636 - val_loss: 0.1934 - val_accuracy: 0.9167
Epoch 2/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1336 - accuracy: 0.9484 - val_loss: 0.2054 - val_accuracy: 0.9162
Epoch 3/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0944 - accuracy: 0.9650 - val_loss: 0.2387 - val_accuracy: 0.9135
Epoch 4/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0702 - accuracy: 0.9752 - val_loss: 0.2693 - val_accuracy: 0.9125
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0535 - accuracy: 0.9796 - val_loss: 0.2673 - val_accuracy: 0.9099
Epoch 6/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0390 - accuracy: 0.9874 - val_loss: 0.3845 - val_accuracy: 0.9026
Epoch 7/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0255 - accuracy: 0.9923 - val_loss: 0.4623 - val_accuracy: 0.8976

Performance and Accuracy

In [74]:
#y_pred = model.predict_classes(X_test)
y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 [==============================] - 1s 4ms/step


In [75]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3131,  288],
       [ 294, 2322]])

In [76]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9035625517812759

Since we are able to see a bit of overfitting hence we will add a Dropout layer

In [86]:
from tensorflow.keras.layers import Dropout
embedding_vector_features = 40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 19s 80ms/step - loss: 0.3426 - accuracy: 0.8407 - val_loss: 0.2240 - val_accuracy: 0.9114
Epoch 2/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1423 - accuracy: 0.9449 - val_loss: 0.2008 - val_accuracy: 0.9135
Epoch 3/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0995 - accuracy: 0.9641 - val_loss: 0.2124 - val_accuracy: 0.9118
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0703 - accuracy: 0.9751 - val_loss: 0.2382 - val_accuracy: 0.9165
Epoch 5/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0513 - accuracy: 0.9824 - val_loss: 0.3065 - val_accuracy: 0.9114
Epoch 6/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0328 - accuracy: 0.9894 - val_loss: 0.3807 - val_accuracy: 0.9100
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0230 - accuracy: 0.9927 - val_loss: 0.4108 - val_accuracy: 0.905

In [87]:
#y_pred = model.predict_classes(y_test)
y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 [==============================] - 1s 3ms/step


In [88]:
confusion_matrix(y_test, y_pred)

array([[3111,  308],
       [ 237, 2379]])

In [90]:
accuracy_score(y_test, y_pred)

0.9096934548467275